In [132]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [133]:
from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api
## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

In [134]:
pip install json_lines

In [135]:
import pandas as pd
import json
import json_lines


## CONTENT
#####################
 
 1. Bali Regency data
 * from Scrapy spider, combined with Wikipedia Data and geojson file from indo.gov.co.id
 2. Indo Province Data (transittion to kawal data - march 2021)
 * kawalCovid data with Kaggle Dataset by Hendra
 3. World - Country Data
 * owid world data

In [136]:
###  -------------------------------------
### 1. COVID DATA BALI REGENCIES
###  -------------------------------------

## From scrapy spider

In [137]:
## Converting Json-Lines File to pandas DataFrame
##########################################

json_file = r'C:\Users\ansve\Coding\Projects-WebScraping\CovidBali\CovidBali\CovidBali\daily.jl'

##Extract data from First TABLE
###############################################

data_first = []
with open(json_file, 'rb') as f:
    count = 0
    for line in json_lines.reader(f, broken=True):
        count+=1
        if count % 2 == 0: #this is the remainder operator
            for x in range(0, 10):
                data_first.append(line['data'][x])

df_first = pd.DataFrame(data_first)
df_first.head()

Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 875 (char 874)',)


,0,1,2,3,4,5,6,7,8,9,10
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020


In [138]:
df_first.tail(30)

,0,1,2,3,4,5,6,7,8,9,10
1110,1,JEMBRANA,0,18,5,2115,0,74,5,2207,08 Apr 2021
1111,2,TABANAN,9,125,6,4016,1,150,16,4291,08 Apr 2021
1112,3,BADUNG,16,275,32,7444,2,204,50,7923,08 Apr 2021
1113,4,DENPASAR,-45,703,108,11984,2,272,65,12959,08 Apr 2021
1114,5,GIANYAR,14,74,6,4566,0,129,20,4769,08 Apr 2021
1115,6,BANGLI,-3,45,2,2001,2,84,1,2130,08 Apr 2021
1116,7,KLUNGKUNG,-4,59,9,1530,1,52,6,1641,08 Apr 2021
1117,8,KARANGASEM,3,86,5,1543,0,86,8,1715,08 Apr 2021
1118,9,BULELENG,-4,187,24,2812,2,128,22,3127,08 Apr 2021
1119,10,KABUPATEN LAINNYA,0,60,10,479,0,7,10,546,08 Apr 2021


In [139]:
## Column names from https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-02.php = df_first and ...-01.php = df_second

columns_first = [
    'No','Kabupaten / County', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases', 'Date']

df_first.columns = columns_first
df_first.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020


In [140]:
################################################
##Extract data from SECOND TABLE
###############################################
data_second = []

with open(json_file, 'rb') as f:
    count =0
    for line in json_lines.reader(f, broken=True):
        count+= 1
        if count % 2 != 0:
            for x in range(0, 10):
                data_second.append(line['data'][x])
df_second = pd.DataFrame(data_second)

Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 875 (char 874)',)


In [141]:
df_second.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12
1145,6,BANGLI,0,59,1,29,12,2067,0,0,13,2155,10 Apr 2021
1146,7,KLUNGKUNG,0,19,0,61,4,1570,0,0,4,1650,10 Apr 2021
1147,8,KARANGASEM,0,23,1,199,0,1495,0,0,1,1717,10 Apr 2021
1148,9,BULELENG,0,43,0,24,12,3087,0,0,12,3154,10 Apr 2021
1149,10,KABUPATEN LAINNYA,0,8,0,135,2,410,0,0,2,553,10 Apr 2021


In [142]:
# # Second data entry
columnName_second =['No', 'Kabupaten / County', 'new PPLN/PMI', 'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)', 'new domestic-travel-history', 'total domestic-travel-history (PPDN)', 'new_local_transmission', 'total_local_transmission',	'new_other_transmission', 'total_other_transmission', 'new_cases' ,'total_cases', 'Date' ]

df_second.columns = columnName_second


In [143]:
df_second = df_second.drop(['Kabupaten / County','new_cases', 'total_cases'], axis=1)

In [144]:
df_second.tail()

,No,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission,Date
1145,6,0,59,1,29,12,2067,0,0,10 Apr 2021
1146,7,0,19,0,61,4,1570,0,0,10 Apr 2021
1147,8,0,23,1,199,0,1495,0,0,10 Apr 2021
1148,9,0,43,0,24,12,3087,0,0,10 Apr 2021
1149,10,0,8,0,135,2,410,0,0,10 Apr 2021


In [145]:
############################
## Merge both df
#############################
df_merged = pd.merge(df_first, df_second, how= 'left', left_on=['No', "Date"], right_on=['No', "Date"])

df_bali = df_merged.copy()
df_bali.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020,0,26,0,11,1,626,0,0
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020,0,21,2,10,23,1466,0,0
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020,0,27,0,4,18,2478,0,0
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020,0,48,-1,22,28,3891,0,0
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020,0,29,-1,10,16,1951,0,0


In [146]:
df = df_bali.copy()
df.tail(93)

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission
1067,8,KARANGASEM,2,67,1,1526,1,86,4,1679,01 Apr 2021,0,23,2,185,2,1471,0,0
1068,9,BULELENG,6,184,23,2677,0,118,29,2979,01 Apr 2021,0,42,0,24,29,2913,0,0
1069,10,KABUPATEN LAINNYA,-16,75,17,425,1,6,2,506,01 Apr 2021,0,7,0,125,2,374,0,0
1070,1,JEMBRANA,-4,22,7,2097,0,73,3,2192,02 Apr 2021,0,26,0,66,3,2100,0,0
1071,2,TABANAN,8,135,8,3933,0,146,16,4214,02 Apr 2021,0,24,2,166,14,4024,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,6,BANGLI,3,52,10,2019,0,84,13,2155,10 Apr 2021,0,59,1,29,12,2067,0,0
1156,7,KLUNGKUNG,-3,53,6,1544,1,53,4,1650,10 Apr 2021,0,19,0,61,4,1570,0,0
1157,8,KARANGASEM,1,88,0,1543,0,86,1,1717,10 Apr 2021,0,23,1,199,0,1495,0,0
1158,9,BULELENG,-16,168,28,2856,0,130,12,3154,10 Apr 2021,0,43,0,24,12,3087,0,0


In [147]:
# delete duplicate columns and convert dtypes
df = df.loc[:,~df.columns.duplicated()]

## fill NaN in Kabubaten Lain fileds (confirmed cases from outside of bali regency but detected in Bali have no match with reference-data e.g)
################################################
df = df.fillna(0)
df[df.isna().any(axis=1)]

## convert columns data types, and capitalize Names
df['No'] = df['No'].astype(int)
df['Name_EN'] = df['Kabupaten / County'].astype(str)
df.replace("KABUPATEN LAIN", "Others", inplace=True)
df['Name_EN'] = df['Name_EN'].str.capitalize()

df.dtypes

No                                                             int32
Kabupaten / County                                            object
new_treatment                                                 object
total_treatment                                               object
new_recovered                                                 object
total_recovered                                               object
new_deaths                                                    object
total_deaths                                                  object
new_cases                                                     object
total_cases                                                   object
Date                                                          object
new PPLN/PMI                                                  object
total history-of-foreign-travel/ migrant-worker (PPLN/PMI)    object
new domestic-travel-history                                   object
total domestic-travel-history (PPD

In [148]:
df.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission,Name_EN
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020,0,26,0,11,1,626,0,0,Jembrana
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020,0,21,2,10,23,1466,0,0,Tabanan
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020,0,27,0,4,18,2478,0,0,Badung
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020,0,48,-1,22,28,3891,0,0,Denpasar
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020,0,29,-1,10,16,1951,0,0,Gianyar


In [149]:
## convert date to datetime 
################################

# from datetime import datetime -> datetime.strptime(date, format)
# df["Date"].apply(lambda x: parse(df['Date']))
# or
# df['Date'] = pd.to_datetime(df['Date'])
## also possible

from dateutil.parser import parse
# check if Nan
# df['Date'].isnull().values

df['Date'] = df['Date'].str.strip()
df['Date'] = df['Date'].astype(str) 
df['Date'] = pd.to_datetime(df.Date, errors='coerce')
df['Date'][0]


Timestamp('2020-12-05 00:00:00')

In [150]:
## Reference Data 
## additional data like population, density etc.

# data cleaning, processing
##########################

# add addtitional data
df_ref = pd.read_excel('Bali_reference_data.xlsx')
df_ref = df_ref.drop(columns=['https://en.wikipedia.org/wiki/Bali#cite_note-BPS2019-2', 'https://sp2010.bps.go.id/index.php/site/tabel?tid=321&wid=0', 'Source'])

## adjust regency names of both data sources before merge
##########################
df_ref['Name_EN'] = df_ref['Name_Indo'].str.capitalize()
df_ref.replace('Kota denpasar', 'Denpasar', inplace=True)
df_ref.replace('totals', 'Bali', inplace=True)
df_ref.replace('Totals', 'Bali', inplace=True)

df_ref.head(15)

,Name_Indo,Name_EN,Capital,area_km2,population_2000,population_2010,population_2015,HDI_2014_estimate,Gouvernor,Nr. of Sub-Districts,Nr of Keluruhan/Villages
0,Bali,Bali,NaN,"5,780.06","3,146,999","3,890,757","4,148,588",0.724 (High),NaN,NaN,NaN
1,Buleleng,Buleleng,Singaraja,"1,364.73",558.181,624.125,645.893,0.691 (Medium),Putu Agus Suradnyana,9.0,19/129
2,Jembrana,Jembrana,Negara,841.80,231.806,261.638,271.423,0.686 (Medium),I Putu Artha,5.0,10/41
3,Karangasem,Karangasem,Amlapura,839.54,360.486,396.487,408.487,0.640 (Medium),I Gusti Ayu Mas Sumatri,8.0,3/75
4,Tabanan,Tabanan,Tabanan,839.30,376.03,420.913,435.753,0.726 (High),Ni Putu Eka Wiryastuti,10.0,-/133
5,Bangli,Bangli,Bangli,490.71,193.776,215.353,222.474,0.657 (Medium),I Made Gianyar,4.0,4/68
6,Badung,Badung,Mangupura,418.52,345.863,543.332,615.148,0.779 (High),I Nyoman Giri Prasta,6.0,16/46
7,Gianyar,Gianyar,Gianyar,368.00,393.155,469.777,494.729,0.742 (High),I Made Agus Mahayastra,7.0,6/64
8,Klungkung,Klungkung,Semarapura,315.00,155.262,170.543,175.573,0.683 (Medium),I Nyoman Suwirta,4.0,6/53
9,Kota Denpasar,Denpasar,Denpasar,127.78,532.44,788.589,879.098,0.816 (Very High),Ida Bagus Rai Dharmawijaya Mantra,4.0,16/27


In [151]:
# merge both df on 'Name_EN'
df_merged = df.merge(df_ref, on="Name_EN", how='left')

In [152]:
df_merged.columns

Index(['No', 'Kabupaten / County', 'new_treatment', 'total_treatment',
       'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths',
       'new_cases', 'total_cases', 'Date', 'new PPLN/PMI',
       'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)',
       'new domestic-travel-history', 'total domestic-travel-history (PPDN)',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission', 'Name_EN',
       'Name_Indo', 'Capital', 'area_km2', 'population_2000',
       'population_2010', 'population_2015', 'HDI_2014_estimate', 'Gouvernor',
       'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages'],
      dtype='object')

In [153]:
# # convert to integers
int_numbers = ['new_treatment', 'total_treatment',
       'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths',
       'new_cases', 'total_cases', 'new PPLN/PMI',
       'total domestic-travel-history (PPDN)', 
       'new domestic-travel-history', 'total domestic-travel-history (PPDN)',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission', 'population_2015']
## check for duplicated columns
# print(df_merged.columns[df_merged.columns.duplicated(keep=False)])
df_merged[int_numbers] = df_merged[int_numbers].apply(pd.to_numeric)
df_merged.tail()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,Name_Indo,Capital,area_km2,population_2000,population_2010,population_2015,HDI_2014_estimate,Gouvernor,Nr. of Sub-Districts,Nr of Keluruhan/Villages
1155,6,BANGLI,3,52,10,2019,0,84,13,2155,...,Bangli,Bangli,490.71,193.776,215.353,222.474,0.657 (Medium),I Made Gianyar,4.0,4/68
1156,7,KLUNGKUNG,-3,53,6,1544,1,53,4,1650,...,Klungkung,Semarapura,315.00,155.262,170.543,175.573,0.683 (Medium),I Nyoman Suwirta,4.0,6/53
1157,8,KARANGASEM,1,88,0,1543,0,86,1,1717,...,Karangasem,Amlapura,839.54,360.486,396.487,408.487,0.640 (Medium),I Gusti Ayu Mas Sumatri,8.0,3/75
1158,9,BULELENG,-16,168,28,2856,0,130,12,3154,...,Buleleng,Singaraja,"1,364.73",558.181,624.125,645.893,0.691 (Medium),Putu Agus Suradnyana,9.0,19/129
1159,10,KABUPATEN LAINNYA,1,65,1,480,0,8,2,553,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
df_merged.population_2015.dtypes

dtype('float64')

In [155]:
# ## add columns with new calculated data
#########################################

df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015'] * 0.1
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_deaths_per_100k'] = df_merged['total_deaths'] / df_merged['population_2015'] * 0.1
df_merged['total_deaths_per_100k'] = df_merged['total_deaths_per_100k'].apply(pd.to_numeric).round(2)

df_merged['new_cases_per_mil'] = df_merged['new_cases'] / df_merged['population_2015'] * 0.1
df_merged['new_cases_per_mil'] = df_merged['new_cases_per_mil'].apply(pd.to_numeric)

df_merged['cases7'] = df_merged['new_cases'].rolling(window=7, center=False).mean()
df_merged['cases7_per_100k'] = df_merged['cases7'] / df_merged['population_2015'] *0.1

df_merged['deaths7'] = df_merged['new_deaths'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['total_deaths_per_100k'] / df_merged['population_2015'] *0.1

# infection fatality ratio (IFR), which estimates this proportion of deaths among all infected individuals. The second is case fatality ratio (CFR), which estimates this proportion of deaths among identified confirmed cases. 
# see https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19
df_merged['CFR'] = df_merged['total_deaths'] / df_merged['total_cases'] * 100
df_merged['CFR'] = df_merged['CFR'].round(2)

## Calculate in App after region selected and df only contains region
df_merged['growth_rate_new_cases'] = df_merged['new_cases'].pct_change(fill_method ='ffill', periods=7)
## Check new created values
##############################
# df_merged[['total_deaths', 'total_cases', 'total_deaths_per_100k', 'population_2015', 'total_cases_per_100k']]

In [156]:
# print(df_merged[])

In [157]:
df_merged[['Name_EN', "total_cases", 'new_cases', 'cases7_per_100k', 'new_cases_per_mil', 'population_2015']][-20:]

,Name_EN,total_cases,new_cases,cases7_per_100k,new_cases_per_mil,population_2015
1140,Jembrana,2208,1,0.003579,0.000368,271.423
1141,Tabanan,4305,14,0.002033,0.003213,435.753
1142,Badung,7976,53,0.002647,0.008616,615.148
1143,Denpasar,13041,82,0.003088,0.009328,879.098
1144,Gianyar,4784,15,0.005689,0.003032,494.729
1145,Bangli,2142,12,0.012008,0.005394,222.474
1146,Klungkung,1646,5,0.014809,0.002848,175.573
1147,Karangasem,1716,1,0.006365,0.000245,408.487
1148,Buleleng,3142,15,0.004048,0.002322,645.893
1149,Kabupaten lainnya,551,5,NaN,NaN,NaN


In [158]:
## add id from geojson file
geojson_bali = 'new_bali_id.geojson'

# open new geojson (with id) file
bali_geo_id = json.load(open(geojson_bali, 'r'))

## get names of regencies
regencies = []
for regency in bali_geo_id['features']:
  regencies.append(regency['properties']['ADM2_EN'])

  # add id column in df with feature.id from geojson
bali_id_map = {}
for regency in bali_geo_id['features']:
  bali_id_map[regency['properties']['ADM2_EN']] = regency['id']
bali_id_map

## for Foreigners or Other Regency add 'id' = 0
df_merged['id'] = df_merged['Name_Indo'].apply(lambda x: bali_id_map[x] if x in bali_id_map else 0)
df_merged.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,total_cases_per_100k,total_deaths_per_100k,new_cases_per_mil,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,...,0.24,0.01,0.000368,NaN,NaN,NaN,0.000004,2.11,NaN,5
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,...,0.34,0.01,0.005737,NaN,NaN,NaN,0.000002,3.14,NaN,9
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,...,0.41,0.01,0.002926,NaN,NaN,NaN,0.000002,2.03,NaN,1
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,...,0.45,0.01,0.003071,NaN,NaN,NaN,0.000001,2.27,NaN,8
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,...,0.40,0.02,0.003032,NaN,NaN,NaN,0.000004,3.77,NaN,4


In [159]:
df_merged.tail()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,total_cases_per_100k,total_deaths_per_100k,new_cases_per_mil,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
1155,6,BANGLI,3,52,10,2019,0,84,13,2155,...,0.97,0.04,0.005843,19.857143,0.008926,0.857143,0.000018,3.90,-0.133333,2
1156,7,KLUNGKUNG,-3,53,6,1544,1,53,4,1650,...,0.94,0.03,0.002278,19.714286,0.011229,0.857143,0.000017,3.21,-0.200000,7
1157,8,KARANGASEM,1,88,0,1543,0,86,1,1717,...,0.42,0.02,0.000245,18.571429,0.004546,0.857143,0.000005,5.01,-0.888889,6
1158,9,BULELENG,-16,168,28,2856,0,130,12,3154,...,0.49,0.02,0.001858,18.428571,0.002853,0.714286,0.000003,4.12,-0.076923,3
1159,10,KABUPATEN LAINNYA,1,65,1,480,0,8,2,553,...,NaN,NaN,NaN,13.857143,NaN,0.428571,NaN,1.45,-0.941176,0


In [160]:
# create csv for Bali Dash App
df_merged.to_csv('bali_regency_data.csv')


In [161]:
# save a version in Kaggle Datasets (upload daily)

df_merged.to_csv(r'C:\Users\ansve\Coding\Kaggle\Datasets\Covid19_Bali\Covid19_Bali_Regencies_Data.csv')

In [162]:
###  -------------------------------------
### 2. INDONESIA COVID DATA 
###  -------------------------------------

## From Kaggel Dataset
# 1. connect to kaggle and get latest dataset


In [163]:
# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
# api.dataset_download_files('hendratno/covid19-indonesia/covid_19_indonesia_time_series_all.csv', unzip = True)
# 

In [164]:
df_indo = pd.read_csv('covid_19_indonesia_time_series_all.csv')

df_indo.tail()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
13009,3/28/2021,ID-SG,Sulawesi Tenggara,4,0,1,3,10249,205,9593,...,122.070311,-4.124689,1.52,3888.88,0.00,77.79,2.00%,93.60%,0.31,0.00
13010,3/28/2021,ID-SA,Sulawesi Utara,5,0,4,1,15351,499,12420,...,124.521240,1.259638,1.89,5810.63,0.00,188.88,3.25%,80.91%,0.26,1.00
13011,3/28/2021,ID-SB,Sumatera Barat,106,2,74,30,31455,682,29611,...,100.465062,-0.850253,19.21,5699.15,0.36,123.57,2.17%,94.14%,0.96,2.00
13012,3/28/2021,ID-SS,Sumatera Selatan,101,3,77,21,17443,828,15450,...,104.169465,-3.216212,12.29,2122.65,0.37,100.76,4.75%,88.57%,1.20,1.50
13013,3/28/2021,ID-SU,Sumatera Utara,69,2,37,30,27120,903,23714,...,99.051964,2.191894,4.64,1823.21,0.13,60.71,3.33%,87.44%,0.86,0.67


In [165]:
## convert dates to datetime format so matching with new dataframe before merging

print(df_indo['Date'][0])
df_indo['Date'] = pd.to_datetime(df_indo['Date'])


3/1/2020


In [166]:
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'New Cases', 'New Deaths',
       'New Recovered', 'New Active Cases', 'Total Cases', 'Total Deaths',
       'Total Recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'Case Fatality Rate', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths'],
      dtype='object')

In [167]:
# adjust column names
columns_new = ['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'CFR', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths']

df_indo.columns = columns_new
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths'],
      dtype='object')

In [168]:
df_indo['Population'].iloc[1]


10846145

In [169]:
## add id from geojson for choropleth Map
#########################################
# open new geojson (with id) file
geojson_indo = 'new_indo_id.geojson'
indo_geo_id = json.load(open(geojson_indo, 'r'))


In [170]:
# indo_geo_id['features'][7]['id']

In [171]:
## ADJUST NAMES OF df_indo with NAMES OF GEOJSON ADM1_EN

# which names are different
names_df = df_indo['Location'].unique()
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

list(set(provinces) - set(names_df))

['Bangka-Belitung', 'Yogyakarta', 'Irian Jaya Barat', 'Jakarta Raya']

In [172]:
list_df = ['DKI Jakarta', 'Kalimantan Utara', 'Daerah Istimewa Yogyakarta', 'Papua Barat','Kepulauan Bangka Belitung']

list(set(provinces) - set(names_df))
list1= ['Bangka-Belitung', 'Jakarta Raya', 'Yogyakarta', 'Irian Jaya Barat']


In [173]:
# adjust Names of provnices in Indo

df_indo = df_indo.replace("DKI Jakarta", 'Jakarta Raya')
df_indo = df_indo.replace("Papua Barat", 'Irian Jaya Barat')
df_indo = df_indo.replace("Daerah Istimewa Yogyakarta", 'Yogyakarta')
df_indo = df_indo.replace("Kepulauan Bangka Belitung", 'Bangka-Belitung')

In [174]:
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths'],
      dtype='object')

In [175]:
df_indo.Location.unique()

array(['Jakarta Raya', 'Indonesia', 'Jawa Barat', 'Riau', 'Banten',
       'Sulawesi Tenggara', 'Kalimantan Timur', 'Yogyakarta',
       'Sumatera Utara', 'Jawa Tengah', 'Jawa Timur', 'Sulawesi Selatan',
       'Jambi', 'Maluku', 'Papua', 'Maluku Utara', 'Aceh',
       'Kalimantan Tengah', 'Lampung', 'Sulawesi Tengah',
       'Sumatera Barat', 'Irian Jaya Barat', 'Kalimantan Utara',
       'Sulawesi Barat', 'Sumatera Selatan', 'Bali', 'Kalimantan Barat',
       'Kalimantan Selatan', 'Bangka-Belitung', 'Kepulauan Riau',
       'Bengkulu', 'Nusa Tenggara Barat', 'Sulawesi Utara',
       'Nusa Tenggara Timur', 'Gorontalo'], dtype=object)

In [176]:
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

# add id column in df with feature.id from geojson
indo_id_map = {}
for province in indo_geo_id['features']:
  indo_id_map[province['properties']['state']] = province['id']
# indo_id_map

# ## for Foreigners or Other Regency add 'id' = 0
df_indo['id'] = df_indo['Location'].apply(lambda x: indo_id_map[x] if x in indo_id_map else 0)

df_indo.tail(100)


,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,CFR,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,id
12914,2021-03-26,ID-YO,Yogyakarta,232,8,140,84,32544,778,26524,...,-7.894502,63.89,8962.78,2.20,214.27,2.39%,81.50%,0.90,8.00,3
12915,2021-03-26,ID-GO,Gorontalo,21,0,13,8,5103,144,4784,...,0.687003,17.79,4322.19,0.00,121.97,2.82%,93.75%,3.00,1.00,26
12916,2021-03-26,IDN,Indonesia,4982,85,5679,-782,1487541,40166,1322878,...,-0.789275,18.79,5609.44,0.32,151.46,2.70%,88.93%,0.82,0.87,0
12917,2021-03-26,ID-JA,Jambi,15,2,10,3,6034,93,4928,...,-1.697698,4.29,1727.28,0.57,26.62,1.54%,81.67%,0.31,NaN,28
12918,2021-03-26,ID-JB,Jawa Barat,855,1,1495,-641,245778,3031,217531,...,-6.920432,18.93,5442.22,0.02,67.11,1.23%,88.51%,0.73,0.08,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13009,2021-03-28,ID-SG,Sulawesi Tenggara,4,0,1,3,10249,205,9593,...,-4.124689,1.52,3888.88,0.00,77.79,2.00%,93.60%,0.31,0.00,23
13010,2021-03-28,ID-SA,Sulawesi Utara,5,0,4,1,15351,499,12420,...,1.259638,1.89,5810.63,0.00,188.88,3.25%,80.91%,0.26,1.00,22
13011,2021-03-28,ID-SB,Sumatera Barat,106,2,74,30,31455,682,29611,...,-0.850253,19.21,5699.15,0.36,123.57,2.17%,94.14%,0.96,2.00,2
13012,2021-03-28,ID-SS,Sumatera Selatan,101,3,77,21,17443,828,15450,...,-3.216212,12.29,2122.65,0.37,100.76,4.75%,88.57%,1.20,1.50,33


## KawalCovid19 Data

## get latest Province Data from 
https://docs.google.com/spreadsheets/d/1ma1T9hWbec1pXlwZ89WakRk-OfVUQZsOCFl4FwZxzVw/edit#gid=2052139453

* process and clean
* merge with df_indo
* update daily


In [177]:
df = pd.read_excel('kawalcovid19.xlsx', sheet_name="Timeline")


In [178]:
# df.head()

In [179]:
## transform excel sheet to timeseries table (columns: Data, Province, total_cases, daily_cases, active_cases etc.)
case_type = ['Total Kasus', 'Kasus Harian', 'Kasus Aktif', 'Sembuh', 'Sembuh Harian', 'Meninggal Dunia', 'Meninggal Dunia Harian', 'Meninggal Dunia Harian']

In [180]:
# extract df for each case-type
##################################

# replace all NaN with 0
# delete empty rows
# adjust header
# extract for each Province, add Province Name and merge, and sort by date

df_totalKasus = df.iloc[:df.loc[df['Total Kasus'].str.contains('Kasus Harian', na=False)].index[0]]
df_totalKasus = df_totalKasus.rename(columns={'Total Kasus': "Date"})
df_totalKasus= df_totalKasus.fillna(0)
df_totalKasus.set_index('Date', inplace=True)
df_totalKasus= df_totalKasus[:-1]
df_totalKasus = df_totalKasus.drop(['?', 'Unnamed: 36'], axis=1)

df_kasusHarian = df.iloc[df.loc[df['Total Kasus'].str.contains('Kasus Harian', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Kasus Aktif', na=False)].index[0]]
df_kasusHarian = df_kasusHarian.rename(columns={'Total Kasus': "Date"})
df_kasusHarian= df_kasusHarian.fillna(0)
df_kasusHarian.set_index('Date', inplace=True)
df_kasusHarian= df_kasusHarian[1:-1]
df_kasusHarian = df_kasusHarian.drop(['?', 'Unnamed: 36'], axis=1)

df_kasusAktif = df.iloc[df.loc[df['Total Kasus'].str.contains('Kasus Aktif', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Sembuh', na=False)].index[0]]
df_kasusAktif = df_kasusAktif.rename(columns={'Total Kasus': "Date"})
df_kasusAktif= df_kasusAktif.fillna(0)
df_kasusAktif.set_index('Date', inplace=True)
df_kasusAktif= df_kasusAktif[1:-1]
df_kasusAktif = df_kasusAktif.drop(['?', 'Unnamed: 36'], axis=1)

df_sembuh = df.iloc[df.loc[df['Total Kasus'].str.contains('Sembuh', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Sembuh Harian', na=False)].index[0]]
df_sembuh = df_sembuh.rename(columns={'Total Kasus': "Date"})
df_sembuh= df_sembuh.fillna(0)
df_sembuh.set_index('Date', inplace=True)
df_sembuh= df_sembuh[1:-1]
df_sembuh = df_sembuh.drop(['?', 'Unnamed: 36'], axis=1)

df_sembuhHarian = df.iloc[df.loc[df['Total Kasus'].str.contains('Sembuh Harian', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Meninggal Dunia', na=False)].index[0]]
df_sembuhHarian = df_sembuhHarian.rename(columns={'Total Kasus': "Date"})
df_sembuhHarian= df_sembuhHarian.fillna(0)
df_sembuhHarian.set_index('Date', inplace=True)
df_sembuhHarian= df_sembuhHarian[1:-1]
df_sembuhHarian = df_sembuhHarian.drop(['?', 'Unnamed: 36'], axis=1)

df_meninggalDunia = df.iloc[df.loc[df['Total Kasus'].str.contains('Meninggal Dunia', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Meninggal Dunia Harian', na=False)].index[0]]
df_meninggalDunia = df_meninggalDunia.rename(columns={'Total Kasus': "Date"})
df_meninggalDunia= df_meninggalDunia.fillna(0)
df_meninggalDunia.set_index('Date', inplace=True)
df_meninggalDunia= df_meninggalDunia[1:-1]
df_meninggalDunia = df_meninggalDunia.drop(['?', 'Unnamed: 36'], axis=1)

df_meninggalDuniaHarian = df.iloc[df.loc[df['Total Kasus'].str.contains('Meninggal Dunia Harian', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Meninggal Dunia Harian', na=False)].index[1]]
df_meninggalDuniaHarian = df_meninggalDuniaHarian.rename(columns={'Total Kasus': "Date"})
df_meninggalDuniaHarian= df_meninggalDuniaHarian.fillna(0)
df_meninggalDuniaHarian.set_index('Date', inplace=True)
df_meninggalDuniaHarian= df_meninggalDuniaHarian[1:-3]
df_meninggalDuniaHarian = df_meninggalDuniaHarian.drop(['?', 'Unnamed: 36'], axis=1)

dfs = [df_totalKasus, df_kasusHarian, df_kasusAktif, df_sembuh, df_sembuhHarian, df_meninggalDunia, df_meninggalDuniaHarian]
dfs_names = ["df_totalKasus", "df_kasusHarian", "df_kasusAktif", "df_sembuh","df_sembuhHarian", "df_meninggalDunia", "df_meninggalDuniaHarian"]
dfs_type = ["totalKasus", "kasusHarian", "kasusAktif", "sembuh","sembuhHarian", "meninggalDunia", "meninggalDuniaHarian"]

In [181]:
column_names = df_totalKasus.columns
column_names

Index(['Aceh', 'Bali', 'Banten', 'Babel', 'Bengkulu', 'DIY', 'Jakarta',
       'Jambi', 'Jabar', 'Jateng', 'Jatim', 'Kalbar', 'Kaltim', 'Kalteng',
       'Kalsel', 'Kaltara', 'Kep Riau', 'NTB', 'Sumsel', 'Sumbar', 'Sulut',
       'Sumut', 'Sultra', 'Sulsel', 'Sulteng', 'Lampung', 'Riau', 'Malut',
       'Maluku', 'Papbar', 'Papua', 'Sulbar', 'NTT', 'Gorontalo'],
      dtype='object')

In [182]:
## First Dataframe - Total Cases per Province

df_totKasus = pd.DataFrame({'Province': [0], "Total Kasus":[0]})
for i, province in enumerate(column_names):
    case_numbers = df_totalKasus[province]
    df_x = pd.DataFrame({'Province': case_numbers, "Total Kasus": case_numbers })
    df_x["Province"] = column_names[i]
    # df_x['Total Kasus'] = df_totalKasus[province]
    if i == 0:
        df_totKasus = df_x
    else:
        df_totKasus = df_totKasus.append(df_x)

In [183]:
print(df_totKasus.shape)
df_totKasus1 = df_totKasus.reset_index()
df_totKasus1.head()

(13192, 2)


,Date,Province,Total Kasus
0,2020-03-18,Aceh,0
1,2020-03-19,Aceh,0
2,2020-03-20,Aceh,0
3,2020-03-21,Aceh,0
4,2020-03-22,Aceh,0


In [184]:
## since memory error split dataframes (only with join)
## learning: if not merge on index -> use .merge() 2) reset_index() get datetime out of index first

dfs1 =dfs[1:]
df_total = pd.DataFrame({"Date": [0], 'Province': [0], dfs_type[0]:[0]})

for index, df in enumerate(dfs1):
    count = index + 1
    df_name = dfs_names[index+1]+'Clean'
    df_name = pd.DataFrame({"Date": [0], 'Province': [0], dfs_type[count]:[0]})
    for i, province in enumerate(column_names):
        case_numbers = df[province]
        df_x = pd.DataFrame({'Province': case_numbers, dfs_type[count]: case_numbers })
        df_x["Province"] = column_names[i]
        if i == 0:
            df_name = df_x
        else:
            df_name = df_name.append(df_x)

    df_name = df_name.reset_index()
    if index == 0:
        df_total = df_name
    else:
        df_total = pd.merge(df_total, df_name, on=['Date', 'Province'])   

In [185]:
## sort_dates
# check if datetime format correct
print(df_total['Date'].iloc[0])
df_total = df_total.sort_values(by="Date")
df_total.head()

2020-03-21 00:00:00


,Date,Province,kasusHarian,kasusAktif,sembuh,sembuhHarian,meninggalDunia,meninggalDuniaHarian
0,2020-03-21,Aceh,0,0,0,0,0,0
5348,2020-03-21,Kalsel,0,0,0,0,0,0
6494,2020-03-21,NTB,0,0,0,0,0,0
11460,2020-03-21,Papua,0,0,0,0,0,0
11842,2020-03-21,Sulbar,0,0,0,0,0,0


In [186]:
# merge with total Kasus df from before
df_total = pd.merge(df_total, df_totKasus1, on=['Date', 'Province'])
df_total.tail() 

,Date,Province,kasusHarian,kasusAktif,sembuh,sembuhHarian,meninggalDunia,meninggalDuniaHarian,Total Kasus
12983,2021-04-07,Maluku,6,359,6921,24,112,0,7392
12984,2021-04-07,Papbar,26,412,7900,26,140,3,8452
12985,2021-04-07,Papua,60,9084,10500,20,204,0,19788
12986,2021-04-07,Kaltara,6,1484,9640,38,178,1,11302
12987,2021-04-07,Gorontalo,16,202,4873,4,150,0,5225


In [187]:
# adjust Province Names
df_total['Province'].unique()

array(['Aceh', 'Kalsel', 'NTB', 'Papua', 'Sulbar', 'Bali', 'Kalteng',
       'Sumsel', 'Papbar', 'Banten', 'Kaltim', 'Sumbar', 'Kalbar',
       'Maluku', 'NTT', 'Kep Riau', 'Sulut', 'Jatim', 'Gorontalo',
       'Sumut', 'Malut', 'Bengkulu', 'Jateng', 'Sultra', 'Jabar',
       'Sulsel', 'Riau', 'DIY', 'Jambi', 'Sulteng', 'Lampung', 'Babel',
       'Kaltara', 'Jakarta'], dtype=object)

In [188]:
# adjust Province Names
indo_slang_prov_names = ['Aceh', 'Bali', 'Maluku', 'Sulbar', 'Malut', 'Banten', 'Riau',
       'Lampung', 'Babel', 'Sulteng', 'Sulsel', 'Bengkulu', 'Sultra',
       'Sumut', 'DIY', 'Papbar', 'Sulut', 'NTT', 'Jakarta', 'Sumsel',
       'NTB', 'Jambi', 'Kep Riau', 'Jabar', 'Kaltara', 'Kalsel',
       'Gorontalo', 'Jateng', 'Kalteng', 'Kaltim', 'Jatim', 'Sumbar',
       'Papua', 'Kalbar']

# from previous Indo-province dataset
correct_prov_names = ['Jakarta Raya', 'Indonesia', 'Jawa Barat', 'Riau', 'Banten',
       'Sulawesi Tenggara', 'Kalimantan Timur', 'Yogyakarta',
       'Sumatera Utara', 'Jawa Tengah', 'Jawa Timur', 'Sulawesi Selatan',
       'Maluku', 'Papua', 'Maluku Utara', 'Aceh', 'Kalimantan Tengah',
       'Lampung', 'Sulawesi Tengah', 'Sumatera Barat', 'Irian Jaya Barat',
       'Kalimantan Utara', 'Sulawesi Barat', 'Sumatera Selatan', 'Bali',
       'Kalimantan Barat', 'Jambi', 'Kalimantan Selatan',
       'Bangka-Belitung', 'Kepulauan Riau', 'Bengkulu',
       'Nusa Tenggara Barat', 'Nusa Tenggara Timur',
       'Gorontalo']


In [189]:

print(list(set(indo_slang_prov_names) - set(correct_prov_names)))

# adjust column names
df_total = df_total.replace({
    'Sultra': 'Sulawesi Utara',
    "Jakarta": 'Jakarta Raya',
    "Kalteng": 'Kalimantan Tengah',
    'Sulsel': 'Sulawesi Selatan',
    'Sulut': 'Sulawesi Utara',
    'Kaltim': "Kalimantan Timur", 'Kalsel': 'Kalimantan Selatan','Malut': 'Maluku Utara','Sumsel': 'Sumatera Barat','NTT': 'Nusa Tenggara Timur','Kalbar': 'Kalimantan Barat','Jateng': 'Jawa Tengah','Kep Riau': 'Kepulauan Riau','Kaltara': 'Kalimantan Utara','Sulbar': 'Sulawesi Barat','Babel': 'Bangka-Belitung','Jatim': "Jawa Timur",'Sumbar': 'Sumatera Barat',"Sumut": 'Sumatera Utara','Jabar': 'Jawa Barat','NTB': 'Nusa Tenggara Barat','DIY': 'Yogyakarta','Sulteng': 'Sulawesi Tengah','Papbar': 'Papua','Jakarta': 'Jakarta Raya'})


['Sumbar', 'Kep Riau', 'Papbar', 'DIY', 'Jakarta', 'Sulteng', 'Jabar', 'Malut', 'Kaltara', 'NTB', 'Sulsel', 'Kaltim', 'Sumsel', 'Kalteng', 'Sumut', 'Kalsel', 'Sultra', 'Sulbar', 'Sulut', 'Kalbar', 'Jateng', 'NTT', 'Babel', 'Jatim']


In [190]:
df_total.columns

Index(['Date', 'Province', 'kasusHarian', 'kasusAktif', 'sembuh',
       'sembuhHarian', 'meninggalDunia', 'meninggalDuniaHarian',
       'Total Kasus'],
      dtype='object')

In [191]:
#adjust column names
new_col_names = ['Date', 'Province', 'new_cases', 'active_cases', 'total_recovered', 'new_recovered', 'total_deaths', 'new_deaths', 'total_cases']
df_total.columns = new_col_names
df_total.tail()

,Date,Province,new_cases,active_cases,total_recovered,new_recovered,total_deaths,new_deaths,total_cases
12983,2021-04-07,Maluku,6,359,6921,24,112,0,7392
12984,2021-04-07,Papua,26,412,7900,26,140,3,8452
12985,2021-04-07,Papua,60,9084,10500,20,204,0,19788
12986,2021-04-07,Kalimantan Utara,6,1484,9640,38,178,1,11302
12987,2021-04-07,Gorontalo,16,202,4873,4,150,0,5225


In [192]:
# Get previous Indo_df with selected columns and merge with df_total Indo Province 
# use only selected columns
cols = ['Location', 'Location Level', 'City or Regency', 'Country', 'Continent',
       'Island', 'Time Zone', 'Special Status', 'Total Regencies',
       'Total Cities', 'Total Districts', 'Total Urban Villages',
       'Total Rural Villages', 'Area (km2)', 'Population',
       'Population Density', 'Longitude', 'Latitude','id']
df_indo_small = df_indo[cols]
df_indo_small.tail()

,Location,Location Level,City or Regency,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,id
13009,Sulawesi Tenggara,Province,NaN,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,15,2.0,219,377.0,1911.0,38068,2635461,69.23,122.070311,-4.124689,23
13010,Sulawesi Utara,Province,NaN,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,11,4.0,171,332.0,1507.0,13892,2641884,190.17,124.521240,1.259638,22
13011,Sumatera Barat,Province,NaN,Indonesia,Asia,Sumatera,UTC+07:00,NaN,12,7.0,179,230.0,928.0,42013,5519245,131.37,100.465062,-0.850253,2
13012,Sumatera Selatan,Province,NaN,Indonesia,Asia,Sumatera,UTC+07:00,NaN,13,4.0,241,387.0,2853.0,91592,8217551,89.72,104.169465,-3.216212,33
13013,Sumatera Utara,Province,NaN,Indonesia,Asia,Sumatera,UTC+07:00,NaN,25,8.0,450,693.0,5417.0,72981,14874889,203.82,99.051964,2.191894,4


In [193]:
print(df_indo_small.shape, df_total.shape)

(13014, 19) (12988, 9)


In [194]:
df_indo_small = df_indo_small.drop_duplicates(subset='Location')
df_indo_small.shape

(35, 19)

In [195]:
# check if same Province Names
print(list(set(df_total.Province.unique()) - set(df_indo_small.Location.unique())))

[]


In [196]:
df_indo_full = pd.merge(df_total, df_indo_small, left_on='Province', right_on='Location')
df_indo_full = df_indo_full.sort_values(by='Date')
df_indo_full.tail()

,Date,Province,new_cases,active_cases,total_recovered,new_recovered,total_deaths,new_deaths,total_cases,Location,...,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,id
9167,2021-04-07,Jawa Tengah,535,31547,132934,2222,7389,2,171870,Jawa Tengah,...,6.0,576,753.0,7809.0,32801,36364072,1108.64,110.201115,-7.259097,30
3818,2021-04-07,Sumatera Barat,70,1430,15892,63,866,6,18188,Sumatera Barat,...,7.0,179,230.0,928.0,42013,5519245,131.37,100.465062,-0.850253,2
3819,2021-04-07,Sumatera Barat,129,1302,30438,89,700,2,32440,Sumatera Barat,...,7.0,179,230.0,928.0,42013,5519245,131.37,100.465062,-0.850253,2
2291,2021-04-07,Sulawesi Barat,1,85,5176,29,114,0,5375,Sulawesi Barat,...,NaN,69,73.0,575.0,16787,1559984,92.93,119.345019,-2.461746,32
12987,2021-04-07,Jakarta Raya,862,6320,375628,626,6390,11,388338,Jakarta Raya,...,5.0,44,267.0,NaN,664,10846145,16334.31,106.836118,-6.204699,27


In [197]:
# ## add additional metrics
#########################################

df_indo_full['total_cases_per_100k'] = df_indo_full['total_cases'] / df_indo_full['Population'] * 100000
df_indo_full['total_cases_per_100k'] = df_indo_full['total_cases_per_100k'].apply(pd.to_numeric)

df_indo_full['total_deaths_per_100k'] = df_indo_full['total_deaths'] / df_indo_full['Population'] * 100000
df_indo_full['total_deaths_per_100k'] = df_indo_full['total_deaths_per_100k'].apply(pd.to_numeric)

df_indo_full['new_cases_per_mil'] = df_indo_full['new_cases'] / df_indo_full['Population'] *1000000
df_indo_full['new_cases_per_mil'] = df_indo_full['new_cases_per_mil'].apply(pd.to_numeric)

df_indo_full['cases7'] = df_indo_full['new_cases'].rolling(window=7, center=False).mean()
df_indo_full['cases7_per_100k'] = df_indo_full['cases7'] / df_indo_full['Population'] * 100000
df_indo_full['cases7_per_100k'] = df_indo_full['cases7_per_100k'].apply(pd.to_numeric)

df_indo_full['deaths7'] = df_indo_full['new_deaths'].rolling(window=7, center=False).mean()
df_indo_full['deaths7_per_100k'] = df_indo_full['total_deaths_per_100k'].rolling(window=7, center=False).mean()
df_indo_full['deaths7_per_100k'] = df_indo_full['deaths7_per_100k'].apply(pd.to_numeric)

df_indo_full['CFR'] = df_indo_full['total_deaths'] / df_indo_full['total_cases'] * 100
df_indo_full['CFR'] = df_indo_full['CFR'].round(2)

## Calculate in App after region selected and df only contains region?
df_indo_full['growth_rate_new_cases'] = df_indo_full['new_cases'].pct_change(fill_method ='ffill', periods=7)

In [198]:
df_indo_full[['new_cases', 'Population', 'new_cases_per_mil', 'cases7_per_100k']]

,new_cases,Population,new_cases_per_mil,cases7_per_100k
0,0,5247257,0.000000,NaN
11842,0,1379767,0.000000,NaN
9932,0,6074100,0.000000,NaN
1146,0,4340348,0.000000,NaN
1147,0,4340348,0.000000,NaN
...,...,...,...,...
9167,535,36364072,14.712324,0.832847
3818,70,5519245,12.682894,5.093864
3819,129,5519245,23.372762,4.884208
2291,1,1559984,0.641032,15.210779


In [199]:
df_indo_full.columns

Index(['Date', 'Province', 'new_cases', 'active_cases', 'total_recovered',
       'new_recovered', 'total_deaths', 'new_deaths', 'total_cases',
       'Location', 'Location Level', 'City or Regency', 'Country', 'Continent',
       'Island', 'Time Zone', 'Special Status', 'Total Regencies',
       'Total Cities', 'Total Districts', 'Total Urban Villages',
       'Total Rural Villages', 'Area (km2)', 'Population',
       'Population Density', 'Longitude', 'Latitude', 'id',
       'total_cases_per_100k', 'total_deaths_per_100k', 'new_cases_per_mil',
       'cases7', 'cases7_per_100k', 'deaths7', 'deaths7_per_100k', 'CFR',
       'growth_rate_new_cases'],
      dtype='object')

In [200]:
df_indo_full.to_csv('indo_province_data.csv')

# 3. World Data 
Comparison Data

https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv

url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"


In [201]:
df_world = pd.read_csv(url)

NameError: name 'url' is not defined

In [49]:
df_world.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [50]:
pd.unique(df_world['location'])

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Asia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Europe', 'European Union', 'Faeroe Islands', 'Falkland Islands',
       'Fiji', 'Finland', 'Fran

In [51]:
df_world.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hun

In [52]:
# df_world['location'].unique()

In [53]:
# relevant countries :
rel_countries = ["Germany", 'Europe', 'World', 'International', 'Indonesia',  'United Kingdom', 'Australia', 'France', "Italy", 'United States']

In [54]:
# CFR, total cases per 100k and total deaths, growth_rate, vaccination
## add columns for total cases/deaths per 100k
df_world['Date'] = df_world['date'] 
df_world['CFR'] = df_world['total_deaths'] / df_world['total_cases'] * 100
df_world['total_cases_per_100k'] = df_world['total_cases_per_million'] / 10
df_world['total_deaths_per_100k'] = df_world['total_deaths_per_million'] / 10
df_world['new_cases_per_mil'] = df_world['new_cases_per_million']


In [55]:
df_germany = df_world[df_world['location'] == 'Germany']
df_germany.head()
df_germany['total_cases_per_100k'].iloc[-1]
# df_germany['total_cases_per_million'].iloc[-1]

3456.9022000000004

In [56]:
df_world.to_csv("world_data.csv")